In [1]:
!pwd

/opt/ml/input/code


In [121]:
!pip install seaborn

     |████████████████████████████████| 288 kB 21.5 MB/s eta 0:00:01
     |████████████████████████████████| 9.4 MB 75.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 71.5 MB/s eta 0:00:01
     |████████████████████████████████| 295 kB 78.0 MB/s eta 0:00:01
     |████████████████████████████████| 965 kB 72.8 MB/s eta 0:00:01


In [122]:
import pandas as pd
import numpy as np
import re
import seaborn as sns

In [217]:
# users = pd.read_csv('./data/users.csv')
books = pd.read_csv("./data/books.csv", encoding='utf-8')
# ratings = pd.read_csv('./data/train_ratings.csv')
# test = pd.read_csv('./data/test_ratings.csv')

In [7]:
users

,user_id,location,age
0,8,"timmins, ontario, canada",NaN
1,11400,"ottawa, ontario, canada",49.0
2,11676,"n/a, n/a, n/a",NaN
3,67544,"toronto, ontario, canada",30.0
4,85526,"victoria, british columbia, canada",36.0
...,...,...,...
68087,278376,"danville, pennsylvania, usa",54.0
68088,278621,"victoria, delaware, canada",74.0
68089,278636,"irvington, alabama, usa",NaN
68090,278659,"vancouver, washington, usa",33.0


# USERS
- AGE column은 삭제합니다

In [23]:
users['user_id'].nunique()

68092

#### AGE COLUMN 삭제

In [24]:
users.isna().sum()

user_id         0
location        0
age         27833
dtype: int64

In [25]:
users = users.drop('age', axis=1)

## LOCATION

In [26]:
# USERS에서 location을 location country, city, state로 나눠줌

users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거

users['location_city'] = users['location'].apply(lambda x: x.split(',')[0].strip())
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1].strip())
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2].strip())

users = users.replace('na', np.nan) #특수문자 제거로 n/a가 na로 바뀌게 되었습니다. 따라서 이를 컴퓨터가 인식할 수 있는 결측값으로 변환합니다.
users = users.replace('', np.nan) # 일부 경우 , , ,으로 입력된 경우가 있었으므로 이런 경우에도 결측값으로 변환합니다.

modify_location = users[(users['location_country'].isna())&(users['location_city'].notnull())]['location_city'].values
location = users[(users['location'].str.contains('seattle'))&(users['location_country'].notnull())]['location'].value_counts().index[0]

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass
    
for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2]

/tmp/ipykernel_12168/1343807041.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거


In [27]:
users.isna().sum()

user_id                0
location               0
location_city        122
location_state      1132
location_country     271
dtype: int64

#### location state, city, country가 na인 애들 날려줌 (왜냐! 몇개없음)

In [28]:
users = users.dropna(subset=['location_state', 'location_city', 'location_country']).reset_index(drop=True)

#### locationcolumn도 날려줌 (분리했으니까)

In [29]:
users = users.drop('location', axis=1)

### location country에서 이상한 애들 확인

In [30]:
min(users['location_country'].apply(len))

1

#### location_country가 이상한 애들은 직접 수정해드렸습니다^^

In [31]:
users[users['location_country'].apply(len)==1]

,user_id,location_city,location_state,location_country
44506,41195,c,c,c


In [72]:
# 너무 더러운데.. 어쩌지
users[users['location_country']=='']

,user_id,location_city,location_state,location_country


In [106]:
country_fix_dict = {'usa': {'oklahoma','districtofcolumbia', 'connecticut', 'worcester', 'aroostook', 'texas',  'kern', 'orangeco', 'unitedstatesofamerica', 'fortbend', 'alachua', 'massachusetts', 'arizona', 'austin', 'hawaii', 'ohio', 'camden', 'arkansas', 'minnesota', 'losestadosunidosdenorteamerica', 'us', 'usanow', 'northcarolina', 'maine', 'colorado','oklahoma', 'alabama', 'anystate', 'districtofcolumbia', 'unitedstaes', 'pender', 'newhampshire', 'unitedstates', 'missouri', 'idaho', 'ca', 'newyork','tennessee', 'stthomasi', 'dc', 'washington', 'illinois', 'california', 'michigan', 'iowa', 'maryland', 'newjersey', 'vanwert', 'oregon'},
                    'uk': {'alderney', 'wales',  'aberdeenshire', 'bermuda', 'nottinghamshire', 'scotland', 'usacurrentlylivinginengland', 'england', 'countycork', 'alderney', 'cambridgeshire', 'middlesex', 'northyorkshire', 'westyorkshire', 'cocarlow', 'sthelena'},
                    'japan': {'okinawa'},
                    'southkorea': {'seoul'},
                    'canada': {'ontario', 'alberta', 'novascotia', 'newfoundland', 'newbrunswick', 'britishcolumbia'},
                    'miyanma': {'burma'},
                    'newzealand': {'auckland', 'nz', 'otago'},
                    'spain': {'andalucia','pontevedra', 'gipuzkoa', 'lleida', 'catalunyaspain', 'galiza', 'espaa'},
                    'germany': {'niedersachsen', 'deutschland'},
                    'brazil': {'disritofederal'},
                    'switzerland': {'lasuisse'},
                    'italy': {'veneziagiulia', 'ferrara', 'italia'},
                    'australia': {'nsw', 'queensland', 'newsouthwales'},
                    'belgium': {'labelgique', 'bergued'},
                    'uruguay': {'urugua'},
                    'panama': {'republicofpanama'}
                   }
country_del_list = ['c', 'space', 'universe', 'unknown', 'quit', 'tdzimi', 'universe', 'tn', 'unknown', 'space', 'c', 'franciscomorazan', 'petrolwarnation', 'ineurope', 'hereandthere', 'faraway']

In [107]:
users

,user_id,location_city,location_state,location_country
0,8,timmins,ontario,canada
1,11400,ottawa,ontario,canada
2,67544,toronto,ontario,canada
3,85526,victoria,britishcolumbia,canada
4,96054,ottawa,ontario,canada
...,...,...,...,...
66785,278376,danville,california,usa
66786,278621,victoria,delaware,canada
66787,278636,irvington,alabama,usa
66788,278659,vancouver,britishcolumbia,canada


In [108]:
del_idx = []
for idx, row in enumerate(users['location_country']):
    for key, value in country_fix_dict.items():
        if row in value:
            users.at[idx, 'location_country'] = key
    if row in country_del_list:
        del_idx.append(idx)

In [109]:
users = users.drop(del_idx, axis=0).reset_index(drop=True)

#### location_state 맛만 보겠습니다

In [118]:
users['location_state'].value_counts()

california         6873
ontario            2504
texas              2504
newyork            2398
england            2230
                   ... 
grancanaria           1
vallsoccidental       1
soria                 1
galway                1
ccp                   1
Name: location_state, Length: 1298, dtype: int64

#### location_city 날립니다
- location_state를 날리고 싶다면 알아서 하세요

In [155]:
# 날릴까 말까 => state를 날리고 싶다면 주석 푸세요
# users = users.drop('location_state', axis=1)
users = users.drop('location_city', axis=1)

In [156]:
users

,user_id,location_state,location_country
0,8,ontario,canada
1,11400,ontario,canada
2,67544,ontario,canada
3,85526,britishcolumbia,canada
4,96054,ontario,canada
...,...,...,...
66785,278376,california,usa
66786,278621,delaware,canada
66787,278636,alabama,usa
66788,278659,britishcolumbia,canada


# BOOKS
- summary : 날리자! 사유는 언어가 다 다른데 이거 모델에 넣고 돌리기 too much 노동 크롤링 하면 안 되니까 할 수 없다..가 의견입니다.
- year_of_publication : float -> int
- category : 보류
- language : isbn 코드로 채워주자
  https://en.wikipedia.org/wiki/List_of_ISBN_registration_groups

In [220]:
# float이라서 int로 바꿔줌
books['year_of_publication'] = books['year_of_publication'].astype(int)

#### Summary 삭제

In [221]:
books = books.drop('summary', axis=1) ; books

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,img_path
0,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],images/0002005018.01.THUMBZZZ.jpg
1,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,['1940-1949'],images/0060973129.01.THUMBZZZ.jpg
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,['Medical'],images/0374157065.01.THUMBZZZ.jpg
3,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,['Fiction'],images/0399135782.01.THUMBZZZ.jpg
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,['History'],images/0425176428.01.THUMBZZZ.jpg
...,...,...,...,...,...,...,...,...,...
149565,067161746X,The Bachelor Home Companion: A Practical Guide...,P.J. O'Rourke,1987,Pocket Books,http://images.amazon.com/images/P/067161746X.0...,en,['Humor'],images/067161746X.01.THUMBZZZ.jpg
149566,0767907566,All Elevations Unknown: An Adventure in the He...,Sam Lightner,2001,Broadway Books,http://images.amazon.com/images/P/0767907566.0...,en,['Nature'],images/0767907566.01.THUMBZZZ.jpg
149567,0884159221,Why stop?: A guide to Texas historical roadsid...,Claude Dooley,1985,Lone Star Books,http://images.amazon.com/images/P/0884159221.0...,NaN,NaN,images/0884159221.01.THUMBZZZ.jpg
149568,0912333022,The Are You Being Served? Stories: 'Camping In...,Jeremy Lloyd,1997,Kqed Books,http://images.amazon.com/images/P/0912333022.0...,en,['Fiction'],images/0912333022.01.THUMBZZZ.jpg


#### category
- category high로 묶어주기
- category null값 book_title, book_author 같을 때 na 아닌 거 있으면 걔로 채워넣어주기

##### category_high

In [222]:
# 일단 category에서 대괄호 밖으로 빼기
books.loc[books[books['category'].notnull()].index, 'category'] = books[books['category'].notnull()]['category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip())
# 소문자로 바꾸기
books['category'] = books['category'].str.lower()
# 카테고리를 좀 더 큰 카테고리로 묶어주자
books['category_high'] = books['category'].copy()

In [223]:
books.loc[books[books['category'].str.contains('history',na=False)].index,'category_high'] = 'history'
categories = ['garden','crafts','physics','adventure','music','fiction','nonfiction','science','science fiction','social','homicide',
 'sociology','disease','religion','christian','philosophy','psycholog','mathemat','agricult','environmental',
 'business','poetry','drama','literary','travel','motion picture','children','cook','literature','electronic',
 'humor','animal','bird','photograph','computer','house','ecology','family','architect','camp','criminal','language','india']

for category in categories:
    books.loc[books[books['category'].str.contains(category,na=False)].index,'category_high'] = category
    
 # 5개 이하인 항목은 others로 묶어주도록 하겠습니다.
category_high_df = pd.DataFrame(books['category_high'].value_counts()).reset_index()
category_high_df.columns = ['category','count']
others_list = category_high_df[category_high_df['count']<5]['category'].values
books.loc[books[books['category_high'].isin(others_list)].index, 'category_high']='others'

##### category null 값 묶기

In [224]:
books['category_new'] = books['category_high'].copy()
books['category_new'] = books['category_new'].fillna('0')

In [225]:
books.isna().sum()

isbn                       0
book_title                 0
book_author                0
year_of_publication        0
publisher                  0
img_url                    0
language               67227
category               68851
img_path                   0
category_high          68851
category_new               0
dtype: int64

In [226]:
tmp = books.groupby(['book_title', 'book_author'])['category_new'].apply(lambda x : ' / '.join(x)).to_frame()

In [255]:
tmp.reset_index(drop=False)['category_new'].value_counts().tail(20)

avarice / chocolate / fiction / fiction / 0 / 0                                        1
pets / biography autobiography                                                         1
health fitness / others                                                                1
0 / 0 / fiction / fiction / others / 0 / nonfiction / 0 / 0 / fiction / fiction / 0    1
0 / fiction / 0 / 0 / children                                                         1
0 / adultery                                                                           1
fiction / ghost stories                                                                1
spiritual life / religion / religion                                                   1
african americans / social                                                             1
0 / humor / humor                                                                      1
fiction / magic                                                                        1
0 / 0 / history / 0 /